## Importing libraries we need

In [1]:
import numpy as np
import pickle
from sklearn.utils import shuffle
import keras
from keras import backend as k
from keras.models import Sequential
from keras.layers import BatchNormalization , Conv2D , LeakyReLU,Conv2DTranspose , Dropout
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint

Using TensorFlow backend.


## Handling input and output Data for Training

In [ ]:
# Load training images
train_images = pickle.load(open("inputs.p", "rb" ))

# Load image labels
labels = pickle.load(open("outputs.p", "rb" ))

# Make into arrays as the neural network wants these
#train_images = np.array(train_images)
#labels = np.array(labels)

augmented_images, augmented_labels=[],[]
for image,label in zip(train_images,labels):
    augmented_images.append(np.array(image))
    augmented_labels.append(np.array(label))
    augmented_images.append(np.array(cv2.flip(image,1)))
    augmented_labels.append(np.expand_dims(np.array(cv2.flip(label,1)),axis=2))
    
augmented_images=np.array(augmented_images)
augmented_labels=np.array(augmented_labels)

# Shuffle images along with their labels, then split into training/validation sets
train_images, labels = shuffle(augmented_images, augmented_labels)
# Validation set size 10% and remainder for training
X_train, X_val, y_train, y_val 
    = train_test_split(train_images,labels, test_size=0.1)    
#setting some parameters
batch_size = 1000
epochs = 10

## Building our model

In [2]:
model = Sequential()
model.add(BatchNormalization(input_shape=(256,256,1)))
#encoder part
model.add(Conv2D(64,(4,4),padding='same',strides=(1,1),name='conv1'))
model.add(LeakyReLU(alpha=0.2))
model.add(Conv2D(64,(4,4),padding='same',strides=(2,2),name='conv2'))
model.add(LeakyReLU(alpha=0.2))
model.add(Conv2D(128,(4,4),padding='same',strides=(2,2),name='conv3'))
model.add(LeakyReLU(alpha=0.2))
model.add(Conv2D(256,(4,4),padding='same',strides=(2,2),name='conv4'))
model.add(LeakyReLU(alpha=0.2))
model.add(Dropout(0.2))
model.add(Conv2D(512,(4,4),padding='same',strides=(2,2),name='conv5'))
model.add(LeakyReLU(alpha=0.2))
model.add(Dropout(0.2))
model.add(Conv2D(512,(4,4),padding='same',strides=(2,2),name='conv6'))
model.add(LeakyReLU(alpha=0.2))
model.add(Dropout(0.2))
model.add(Conv2D(512,(4,4),padding='same',strides=(2,2),name='conv7'))
model.add(LeakyReLU(alpha=0.2))
model.add(Dropout(0.2))
model.add(Conv2D(512,(4,4),padding='same',strides=(2,2),name='conv8'))
model.add(LeakyReLU(alpha=0.2))
model.add(Dropout(0.2))
#decoder part
model.add(Conv2DTranspose(512,(4,4),padding='same',strides=(2,2),activation='relu',name='deconv1'))
model.add(Dropout(0.2))
model.add(Conv2DTranspose(512,(4,4),padding='same',strides=(2,2),activation='relu',name='deconv2'))
model.add(Dropout(0.2))
model.add(Conv2DTranspose(512,(4,4),padding='same',strides=(2,2),activation='relu',name='deconv3'))
model.add(Dropout(0.2))
model.add(Conv2DTranspose(256,(4,4),padding='same',strides=(2,2),activation='relu',name='deconv4'))
model.add(Dropout(0.2))
model.add(Conv2DTranspose(128,(4,4),padding='same',strides=(2,2),activation='relu',name='deconv5'))
model.add(Dropout(0.2))
model.add(Conv2DTranspose(64,(4,4),padding='same',strides=(2,2),activation='relu',name='deconv6'))
model.add(Dropout(0.2))
model.add(Conv2DTranspose(64,(4,4),padding='same',strides=(2,2),activation='relu',name='deconv7'))
#last layer
model.add(Conv2D(3,(4,4),padding='same',strides=(1,1),activation='tanh',name='last_layer'))
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_1 (Batch (None, 256, 256, 1)       4         
_________________________________________________________________
conv1 (Conv2D)               (None, 256, 256, 64)      1088      
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 256, 256, 64)      0         
_________________________________________________________________
conv2 (Conv2D)               (None, 128, 128, 64)      65600     
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 128, 128, 64)      0         
_________________________________________________________________
dropout_1 (Dropout)  

## Compiling model and train it

In [ ]:
# compile our model
model.compile(loss='mean_squared_error', optimizer='Adam', metrics=['accuracy'])
#save our model each epoch
ModelCheckpoint(filepath = 'movieColor.h5')
# training our model
model.fit_generator(ImageDataGenerator().flow(X_train, y_train, batch_size=batch_size),steps_per_epoch = len(X_train)/batch_size,validation_data = ImageDataGenerator().flow(X_val, y_val, batch_size=250) 
                    ,validation_steps = len(X_val)/250 ,epochs = 10,verbose=2)